Step 1: create DB on master instance (run this one on the SQL kernel)

In [2]:

USE master
IF EXISTS(select * from sys.databases where name='TwitterData')
DROP DATABASE TwitterData;
GO
CREATE DATABASE TwitterData;
GO
USE TwitterData;
GO


Step 2: switch to scala context and import packages


In [5]:
import java.io.{BufferedReader, File, FileNotFoundException, InputStream, InputStreamReader}
import java.net.URLEncoder
import java.nio.charset.StandardCharsets
import java.util.Base64
import javax.crypto.Mac
import javax.crypto.spec.SecretKeySpec
import scala.collection.JavaConverters._
import org.apache.commons.io.IOUtils
import org.apache.http.client.methods.HttpGet
import org.apache.http.impl.client.CloseableHttpClient
import org.apache.http.impl.client.HttpClients
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.FileSystem
import org.apache.hadoop.fs.Path
import java.io.PrintWriter
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
35,application_1568670927228_0038,spark,idle,Link,Link,✔


SparkSession available as 'spark'.


import java.io.{BufferedReader, File, FileNotFoundException, InputStream, InputStreamReader}
import java.net.URLEncoder
import java.nio.charset.StandardCharsets
import java.util.Base64
import javax.crypto.Mac
import javax.crypto.spec.SecretKeySpec
import scala.collection.JavaConverters._
import org.apache.commons.io.IOUtils
import org.apache.http.client.methods.HttpGet
import org.apache.http.impl.client.CloseableHttpClient
import org.apache.http.impl.client.HttpClients
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.FileSystem
import org.apache.hadoop.fs.Path
import java.io.PrintWriter
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}


setup stuff



In [6]:
//Twitter api auth keys
val consumerKey = "7SMW41Vb3dLfZkWsFYxGa8Mdo"
val consumerSecret = "08pEiMTXY0xyv1TDfUwBYJiNzEOupoVwaw9Vk3tzt1gvsZq9QF"
val accessToken = "10022462-hx6lkyAGKLnGU8GhOHpIUfyRpGhosiph491oY7J6k"
val accessTokenSecret = "avduKmDt6dlL8GzWLRCorTCLTFrAydr43GsjGPHQn3fhT"

//SQL auth stuff
val user = "sa"
val password = "fooRiuzg54"

//Spark-SQL connector params
val hostname = "master-0.master-svc"
val port = 1433
val database = "TwitterData"
val url = s"jdbc:sqlserver://${hostname}:${port};database=${database};user={$user};password=${password};"
val dbtable = "Tweets"
val datasource_name = "TweetsDataSource"

//Twitter stream object parameters

val filters = Array("goose", "springsteen", "punk", "hypnospace")  
val path = "/user/twitter/"
val savingInterval = 2000








consumerKey: String = 7SMW41Vb3dLfZkWsFYxGa8Mdo
consumerSecret: String = 08pEiMTXY0xyv1TDfUwBYJiNzEOupoVwaw9Vk3tzt1gvsZq9QF
accessToken: String = 10022462-hx6lkyAGKLnGU8GhOHpIUfyRpGhosiph491oY7J6k
accessTokenSecret: String = avduKmDt6dlL8GzWLRCorTCLTFrAydr43GsjGPHQn3fhT
user: String = sa
password: String = fooRiuzg54
hostname: String = master-0.master-svc
port: Int = 1433
database: String = TwitterData
url: String = jdbc:sqlserver://master-0.master-svc:1433;database=TwitterData;user={sa};password=fooRiuzg54;
dbtable: String = Tweets
datasource_name: String = TweetsDataSource
filters: Array[String] = Array(goose, springsteen, punk, hypnospace)
path: String = /user/twitter/
savingInterval: Int = 2000


In [7]:
//this is test stuff, don't run it unless you started the stream already
twitterStream.isDownloading

<console>:43: error: not found: value twitterStream
       twitterStream.isDownloading
       ^



In [8]:

class TwitterStream(
  consumerKey: String,
  consumerSecret: String,
  accessToken: String,
  accessTokenSecret: String,
  path: String,
  savingInterval: Long,
  filters: Array[String]) {
  
  private val threadName = "tweet-downloader"
  
  {
    val hasActiveStream = Thread.getAllStackTraces().keySet().asScala.map(_.getName).contains(threadName)
    if (hasActiveStream) {
      throw new RuntimeException(
        "There is already an active stream that writes tweets to the configured path. " +
        "Please stop the existing stream first (using twitterStream.stop()).")
    }
  }
  
  @volatile private var thread: Thread = null
  @volatile private var isStopped = false
  @volatile var isDownloading = false
  @volatile var exception: Throwable = null

  private var httpclient: CloseableHttpClient = null
  private var input: InputStream = null
  private var httpGet: HttpGet = null
  
  private def encode(string: String): String = {
    URLEncoder.encode(string, StandardCharsets.UTF_8.name)
  }

  def start(): Unit = synchronized {
    isDownloading = false
    isStopped = false
    thread = new Thread(threadName) {
      override def run(): Unit = {
        httpclient = HttpClients.createDefault()
        try {
          requestStream(httpclient)
        } catch {
          case e: Throwable => exception = e
        } finally {
          //TwitterStream.this.stop()
        }
      }
    }
    thread.start()
  }

  def requestStream(httpclient: CloseableHttpClient): Unit = {
    val url = "https://stream.twitter.com/1.1/statuses/filter.json"
    val timestamp = System.currentTimeMillis / 1000
    val nonce = timestamp + scala.util.Random.nextInt
    val oauthNonce = nonce.toString
    val oauthTimestamp = timestamp.toString

    val oauthHeaderParams = List(
      "oauth_consumer_key" -> encode(consumerKey),
      "oauth_signature_method" -> encode("HMAC-SHA1"),
      "oauth_timestamp" -> encode(oauthTimestamp),
      "oauth_nonce" -> encode(oauthNonce),
      "oauth_token" -> encode(accessToken),
      "oauth_version" -> "1.0"
    )
    val requestParams = List(
      "track" -> encode(filters.mkString(","))
    )

    val parameters = (oauthHeaderParams ++ requestParams).sortBy(_._1).map(pair => s"""${pair._1}=${pair._2}""").mkString("&")
    val base = s"GET&${encode(url)}&${encode(parameters)}"
    val oauthBaseString: String = base.toString
    val signature = generateSignature(oauthBaseString)
    val oauthFinalHeaderParams = oauthHeaderParams ::: List("oauth_signature" -> encode(signature))
    val authHeader = "OAuth " + ((oauthFinalHeaderParams.sortBy(_._1).map(pair => s"""${pair._1}="${pair._2}"""")).mkString(", "))

    httpGet = new HttpGet(s"https://stream.twitter.com/1.1/statuses/filter.json?${requestParams.map(pair => s"""${pair._1}=${pair._2}""").mkString("&")}")
    httpGet.addHeader("Authorization", authHeader)
    println("Downloading tweets!")
    val response = httpclient.execute(httpGet)
    val entity = response.getEntity()
    input = entity.getContent()
    if (response.getStatusLine.getStatusCode != 200) {
      throw new RuntimeException(IOUtils.toString(input, StandardCharsets.UTF_8))
    }
    isDownloading = true
    val reader = new BufferedReader(new InputStreamReader(input, StandardCharsets.UTF_8))
    var line: String = null
    var lineno = 1
    line = reader.readLine()
    var lastSavingTime = System.currentTimeMillis()
    val s = new StringBuilder()
   
    val conf = new Configuration()
    val fs= FileSystem.get(conf)
                                                                      
    while (line != null && !isStopped) {

      
      lineno += 1
      line = reader.readLine()
      s.append(line + "\n")
      val now = System.currentTimeMillis()
      if (now - lastSavingTime >= savingInterval) {

       
          
         val df = spark.read.json(spark.sparkContext.parallelize(Seq(s.toString)))
         df.write.json(path + now.toString)
          
        lastSavingTime = now
        s.clear()
      }
    }
  }

  private def generateSignature(data: String): String = {
    val mac = Mac.getInstance("HmacSHA1")
    val oauthSignature = encode(consumerSecret) + "&" + encode(accessTokenSecret)
    val spec = new SecretKeySpec(oauthSignature.getBytes, "HmacSHA1")
    mac.init(spec)
    val byteHMAC = mac.doFinal(data.getBytes)
    return Base64.getEncoder.encodeToString(byteHMAC)
  }

  def stop(): Unit = synchronized {
    isStopped = true
    isDownloading = false
    try {
      if (httpGet != null) {
        httpGet.abort()
        httpGet = null
      }
      if (input != null) {
        input.close()
        input = null
      }
      if (httpclient != null) {
        httpclient.close()
        httpclient = null
      }
      if (thread != null) {
        thread.interrupt()
        thread = null
      }
    } catch {
      case _: Throwable =>
    }
  }
}
println("class defined")

defined class TwitterStream
class defined


In [9]:
val twitterStream = new TwitterStream(consumerKey, consumerSecret, accessToken, accessTokenSecret, path, savingInterval, filters)

In [10]:
twitterStream.start()

if (twitterStream.exception != null) { throw twitterStream.exception }

In [11]:
val tweets = spark.read.json(path + "*")
val tweets_schema = tweets.schema

val tweetStream = spark.readStream.
|schema(tweets_schema).
|json(path + "*").
|filter($"lang" === "en").
|withColumn("screen_name", $"user.screen_name").
|withColumn("num_followers", $"user.followers_count").
|withColumn("createdAt", from_utc_timestamp(from_unixtime(unix_timestamp($"created_at", "EEE MMM dd HH:mm:ss ZZZZ yyyy")),"EST")).
|select("screen_name","createdAt","num_followers", "text")

tweets: org.apache.spark.sql.DataFrame = [created_at: string, display_text_range: array<bigint> ... 29 more fields]
tweets_schema: org.apache.spark.sql.types.StructType = StructType(StructField(created_at,StringType,true), StructField(display_text_range,ArrayType(LongType,true),true), StructField(entities,StructType(StructField(hashtags,ArrayType(StringType,true),true), StructField(symbols,ArrayType(StringType,true),true), StructField(urls,ArrayType(StructType(StructField(display_url,StringType,true), StructField(expanded_url,StringType,true), StructField(indices,ArrayType(LongType,true),true), StructField(url,StringType,true)),true),true), StructField(user_mentions,ArrayType(StructType(StructField(id,LongType,true), StructField(id_str,StringType,true), StructField(indices,ArrayType(LongType,true),true), StructField(name,StringType,true), StructField(screen_name,StringType,true)),true),true)),true), ...tweetStream: org.apache.spark.sql.DataFrame = [screen_name: string, createdAt: times

In [12]:

val query = tweetStream.writeStream.outputMode("append").foreachBatch{ (batchDF: DataFrame, batchId: Long) => 
                batchDF.write
                    .format("com.microsoft.sqlserver.jdbc.spark")
                    .mode("append")
                    .option("url", url)
                    .option("dbtable", dbtable)
                    .option("user", user)
                    .option("password", password)
                    .option("dataPoolDataSource",datasource_name).save()
               }.start()
query.processAllAvailable()
//query.awaitTermination(40000)

query: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@33787315


In [0]:
def df_read(dbtable: String,
                url: String,
                dataPoolDataSource: String=""): DataFrame = {
                spark.read
                        .format("com.microsoft.sqlserver.jdbc.spark")
                        .option("url", url)
                        .option("dbtable", dbtable)
                        .option("user", user)
                        .option("password", password)
                        .option("dataPoolDataSource", datasource_name)
                        .load()
                }

In [0]:

val new_df = df_read(dbtable, url, dataPoolDataSource=datasource_name)
new_df.count

In [13]:

twitterStream.stop()

An error was encountered:
Invalid status code '404' from https://172.23.25.58:30443/gateway/default/livy/v1/sessions/35 with error payload: {"msg":"Session '35' not found."}
